In [1]:
import pandas as pd

In [3]:
df = pd.read_csv("../../data/raw/breast-cancer.csv")

df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [26]:
def simple_analysis(df):
    if isinstance(df, pd.DataFrame):
        print("1 The number of instances of the dataframe is # {}".format(df.shape[0]).capitalize())
        print("2 The number of features of the dataframe is # {}".format(df.shape[1]).capitalize())

        if df.isnull().sum().sum() > 0:
            print(
                "3.1 The dataframe has missing values and columns with {}".format(
                    list(df.isnull().sum()[df.isnull().sum() > 0])
                )
            )

        else:
            print("3.2# The dataframe has no missing values".capitalize())

        print("4.1 The dataframe has object type columns: {}".format(df.dtypes[df.dtypes == "object"].index))
        print(
            "4.2 The dataframe has object type columns: {}".format(
                df.dtypes[df.dtypes != "object"].index
            )
        )


if __name__ == "__main__":
    simple_analysis(df)

1 the number of instances of the dataframe is # 569
2 the number of features of the dataframe is # 32
3.2# the dataframe has no missing values
4.1 The dataframe has object type columns: Index(['diagnosis'], dtype='object')
4.2 The dataframe has object type columns: Index(['id', 'radius_mean', 'texture_mean', 'perimeter_mean', 'area_mean',
       'smoothness_mean', 'compactness_mean', 'concavity_mean',
       'concave points_mean', 'symmetry_mean', 'fractal_dimension_mean',
       'radius_se', 'texture_se', 'perimeter_se', 'area_se', 'smoothness_se',
       'compactness_se', 'concavity_se', 'concave points_se', 'symmetry_se',
       'fractal_dimension_se', 'radius_worst', 'texture_worst',
       'perimeter_worst', 'area_worst', 'smoothness_worst',
       'compactness_worst', 'concavity_worst', 'concave points_worst',
       'symmetry_worst', 'fractal_dimension_worst'],
      dtype='object')


In [27]:
df.drop(["id"], axis = 1, inplace = True)

df.head()

,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [28]:
X = df.iloc[:, 1:]
y = df.iloc[:, 0]

In [32]:
import joblib

def dump(value = None, filename = None):
    if (value is not None) and (filename is not None):
        joblib.dump(value=value, filename=filename)
        
    else:
        raise ValueError("The value and filename are required".capitalize())
    
    
def load(filename = None):
    if filename is not None:
        return joblib.load(filename=filename)
    
    else:
        raise ValueError("The filename is required".capitalize())

In [ ]:
import os
PROCESSED_PATH = "../../data/processed/"

In [60]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import os
from joblib import dump


class Loader:
    def __init__(self, dataset=None, batch_size=64, split_size=0.25):
        self.datframe = dataset
        self.batch_size = batch_size
        self.split_size = split_size

    def normalized_dataset(self, dataset):
        if isinstance(dataset, torch.Tensor):
            scaler = StandardScaler()
            return torch.tensor(scaler.fit_transform(dataset))
        else:
            raise TypeError("The dataset must be a torch tensor")

    def preprocessing(self):
        self.dataset = pd.read_csv(self.datframe)

        if isinstance(self.dataset, pd.DataFrame):
            if "id" in self.dataset.columns:
                self.dataset.drop(["id"], axis=1, inplace=True)
            else:
                print("The dataset does not have an id column")

            if "diagnosis" in self.dataset.columns:
                if (
                    "M" in self.dataset["diagnosis"].unique()
                    and "B" in self.dataset["diagnosis"].unique()
                ):
                    self.dataset["diagnosis"] = self.dataset["diagnosis"].map(
                        {"B": 0, "M": 1}
                    )
                else:
                    print("The dataset does not have the correct diagnosis values")
            else:
                print("The dataset does not have a diagnosis column")

            X = self.dataset.iloc[:, 1:]
            y = self.dataset.iloc[:, 0]

            y = y.astype("int32")

            X = torch.tensor(data=X.values, dtype=torch.float32)
            y = torch.tensor(data=y.values, dtype=torch.long)

            X = self.normalized_dataset(dataset=X)

            return {"X": X, "y": y}

        else:
            raise TypeError("The dataset must be a pandas DataFrame")

    def create_dataloader(self):
        data = self.preprocessing()

        if isinstance(data["X"], torch.Tensor) and isinstance(data["y"], torch.Tensor):
            X = data["X"]
            y = data["y"]

            X_train, X_test, y_train, y_test = train_test_split(
                X, y, test_size=self.split_size, random_state=42
            )

            train_dataloader = DataLoader(
                dataset=list(zip(X_train, y_train)),
                batch_size=self.batch_size,
                shuffle=True,
            )

            test_dataloader = DataLoader(
                dataset=list(zip(X_test, y_test)),
                batch_size=self.batch_size,
                shuffle=True,
            )

            os.makedirs(PROCESSED_PATH, exist_ok=True)
            for filename, value in [
                ("train_dataloader", train_dataloader),
                ("test_dataloader", test_dataloader),
            ]:
                dump(
                    value=value,
                    filename=os.path.join(PROCESSED_PATH, f"{filename}.pkl"),
                )

            print(
                "The dataloader has been saved in the folder {}".format(PROCESSED_PATH)
            )

    @staticmethod
    def dataset_details():
        if os.path.exists(PROCESSED_PATH):
            train_dataloader = load(filename=os.path.join(PROCESSED_PATH, "train_dataloader.pkl"))
            test_dataloader = load(filename=os.path.join(PROCESSED_PATH, "test_dataloader.pkl"))

            X, y = next(iter(train_dataloader))

        pd.DataFrame(
            {
                "Train Data(Total)": sum(X.size(0) for X, y in train_dataloader),
                "Test Data(Total)": sum(X.size(0) for X, y in test_dataloader),
                "Data Shpe (Train)": str(X.size()),
            },
            index = ["Quantity"],
            
        ).T.to_csv(
            os.path.join(PROCESSED_PATH, "dataset_details.csv"),
        )


if __name__ == "__main__":
    loader = Loader(
        dataset="../../data/raw/breast-cancer.csv", split_size=0.20, batch_size=64
    )

    loader.create_dataloader()
    
    loader.dataset_details()

The dataloader has been saved in the folder ./processed_data


In [53]:
if ("M" and "B") in df.loc[:, "diagnosis"].unique():
    print(1)

1


In [42]:
"diagnosis" in df.columns

True